# Transfer Learning Model with VGG16 For Fruits and Vegetables Classification

### Set Up Enviroment

In [2]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [3]:
# initial imports
import splitfolders
import os
import shutil

In [4]:
# downloading the dataset
os.environ["KAGGLE_USERNAME"] = "paihwai"
os.environ["KAGGLE_KEY"] = "59919fd8d350a08f43adeb0041ead3f9"

In [5]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73031 sha256=4401603190468e1009ee39d07404c7094a761e086cbccdb1a127d280fe3bd9b1
  Stored in directory: /home/ec2-user/.cache/pip/wheels/fb/01/d5/daa97ea9d89ceec9c6c76fad6fe97165661fd2236c46f83354
Successfully built kaggle


In [6]:
!kaggle datasets download raghavrpotdar/fresh-and-stale-images-of-fruits-and-vegetables

100%|█████████████████████████████████████▉| 1.41G/1.41G [01:09<00:00, 19.8MB/s]
100%|██████████████████████████████████████| 1.41G/1.41G [01:09<00:00, 21.8MB/s]


In [7]:
!unzip fresh-and-stale-images-of-fruits-and-vegetables.zip

Archive:  fresh-and-stale-images-of-fruits-and-vegetables.zip
  inflating: ImageLabels.txt         
  inflating: fresh_apple/Screen Shot 2018-06-08 at 4.59.36 PM.png  
  inflating: fresh_apple/Screen Shot 2018-06-08 at 4.59.49 PM.png  
  inflating: fresh_apple/Screen Shot 2018-06-08 at 4.59.57 PM.png  
  inflating: fresh_apple/Screen Shot 2018-06-08 at 5.00.03 PM.png  
  inflating: fresh_apple/Screen Shot 2018-06-08 at 5.00.12 PM.png  
  inflating: fresh_apple/Screen Shot 2018-06-08 at 5.00.18 PM.png  
  inflating: fresh_apple/Screen Shot 2018-06-08 at 5.00.26 PM.png  
  inflating: fresh_apple/Screen Shot 2018-06-08 at 5.00.35 PM.png  
  inflating: fresh_apple/Screen Shot 2018-06-08 at 5.00.43 PM.png  
  inflating: fresh_apple/Screen Shot 2018-06-08 at 5.00.50 PM.png  
  inflating: fresh_apple/Screen Shot 2018-06-08 at 5.01.01 PM.png  
  inflating: fresh_apple/Screen Shot 2018-06-08 at 5.01.08 PM.png  
  inflating: fresh_apple/Screen Shot 2018-06-08 at 5.01.29 PM.png  
  inflating: fre

In [8]:
os.mkdir('fruitsandvegetables') # make a new folder

In [9]:
# move all the image subfolders into the new folder 
directory = 'fruitsandvegetables'
for i in os.listdir():
    if (i.startswith("fresh") or i.startswith("stale")) and (i.endswith(".zip") == False):
        shutil.move(i, directory)

In [10]:
# pre splitting image data using splitfolders
os.mkdir("TLcnn")
input_path = "fruitsandvegetables"

splitfolders.ratio(input_path, # The location of dataset
                   output="TLcnn", # The output location
                   seed=42, # The number of seed
                   ratio=(.8, .1, .1), # The ratio of splited dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )

Copying files: 14682 files [00:02, 5645.04 files/s]


### Importing Main Libraries

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-02-03 04:46:09.934035: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-03 04:46:33.932898: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [15]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image

from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path

from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

In [16]:
import numpy as np
from matplotlib import pyplot as plt

Matplotlib is building the font cache; this may take a moment.


### Preprocessing

In [29]:
tf.keras.backend.clear_session()

In [30]:
# Create the ImageDataGenerator object
train_datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
) 

test_datagen = ImageDataGenerator(rescale=1./255)

In [31]:
train_set = train_datagen.flow_from_directory(
    directory= "./TLcnn/train",
    target_size= (255,255),
    color_mode="rgb",
    batch_size=64,
    class_mode="categorical", 
    shuffle=True,
    seed=8  
)

Found 11739 images belonging to 12 classes.


In [32]:
val_set = val_datagen.flow_from_directory(
    directory= "./TLcnn/val",
    target_size= (255,255),
    color_mode="rgb",
    batch_size=64,
    class_mode="categorical", 
    shuffle=True,
    seed=8  
)

Found 1464 images belonging to 12 classes.


In [33]:
test_set = test_datagen.flow_from_directory(
    directory= "./TLcnn/test",
    target_size= (255, 255),
    color_mode="rgb",
    batch_size=64,
    class_mode="categorical", 
    shuffle=False
)

Found 1479 images belonging to 12 classes.


In [34]:
train_path = "./TLcnn/train"
test_path = "./TLcnn/test"
val_path = "./TLcnn/val"

In [35]:
## Setting batch size ( later will help with steps per epochs when compiling model)

STEP_SIZE_TRAIN=train_set.n//train_set.batch_size
STEP_SIZE_VALID=val_set.n//val_set.batch_size
STEP_SIZE_TEST=test_set.n//test_set.batch_size

### Get VGG16 Model

In [36]:
tf.keras.backend.set_image_data_format('channels_last')
tf.keras.backend.image_data_format()

'channels_last'

In [37]:
# weights are directly imported from image net
# setting include_top to false will use our own inputs and discard the weights of the input and output layers
# def getVGG16model(LastFourTrainable=False):
vgg_model = VGG16(weights="imagenet", input_shape=(255,255,3), include_top=False)

In [38]:
# Not training any existing weights
for layer in vgg_model.layers[:]:
    layer.trainable = False

Loaded the VGG16 model and froze the pre trained layers. Freezing the layers means the weights cannot be adjusted and it is not trainable. 

## Fine Tune Model

Added my own layers, which is the flattening and prediction layer. 
And then made the model and compiled. 

In [39]:
x = Flatten()(vgg_model.output)
prediction = Dense(12, activation='softmax')(x)
model = Model(inputs=vgg_model.input, outputs=prediction)
model.compile(loss='categorical_crossentropy',
                    optimizer=Adam(),
                    metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 255, 255, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 255, 255, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 255, 255, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 127, 127, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 127, 127, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 127, 127, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 63, 63, 128)       0     

When looking at the model summary, only the last layer (dense layer) is trainable. So the model is only training on the last dense layer. I am only training to test how well the pre trained model detects/classify compared to my base model. 

I have added some hyperparameters like ReduceLRonPlateau. What this hyperparameter does is reduce the learning rate. 
When the model reaches a plateau it reduces the learning rate which helps find the optimal learning rate.
The ModelCheckpoint would be more impactful if I ran a higher epochs, and what this hyperparameter does is it checkpoints at each epochs and picks the best model. But since we are runing only two epochs, this hyperparameter
may not showcase it's function and value as it potentially can. Due to how long it takes for each epochs to run, we are running on a smaller epochs. But since we are also just training on the last layer, it is still sufficent to run just 2 epochs. 

In [40]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=1, save_best_only=True)
callbacks = [checkpoint, lr_reducer]
start = datetime.now()
history = model.fit(train_set,
                    epochs = 2, verbose=2, 
                    validation_data = val_set,
                   )

duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/2
184/184 - 2515s - loss: 0.6753 - accuracy: 0.7754 - val_loss: 0.3104 - val_accuracy: 0.9010 - 2515s/epoch - 14s/step
Epoch 2/2
184/184 - 2536s - loss: 0.2972 - accuracy: 0.8943 - val_loss: 0.2333 - val_accuracy: 0.9133 - 2536s/epoch - 14s/step
Training completed in time:  1:24:12.098771


In [41]:
model.save("Transfer_Learning_Model.h5")  # save the model to the notebook instance

### Model Evaluation

In [42]:
model.evaluate(test_set)

24/24 [==============================] - 275s 11s/step - loss: 0.4977 - accuracy: 0.8600


[0.497739315032959, 0.860040545463562]